In [1]:
from pulp import *

#SETS
customers = [1, 2, 3, 4, 5]
facility = ['f1', 'f2', 'f3']

#PARAMS

demand = {1:80, 2: 270, 3:250, 4: 160, 5: 180}

activation_cost = {'f1': 1000, 'f2': 1000, 'f3': 1000}

capacity = {"f1": 500, "f2": 500, "f3": 500}

distance = {"f1": {1: 4, 2: 5, 3: 6, 4: 8, 5: 10},
            "f2": {1: 6, 2: 4, 3: 3, 4: 5, 5: 8},
            "f3": {1: 9, 2: 7, 3: 4, 4: 3, 5: 4}}

prob = LpProblem("DataPlacement", LpMinimize)
serv_vars = LpVariable.dicts("Service",
                         [(i, j) for i in customers for j in facility], 0) #lower band, >=0


use_vars = LpVariable.dicts("UseLocation", facility, 0, 1, LpBinary)

# OBJECTIVE FUNCTION
prob += lpSum(activation_cost[j]*use_vars[j] for j in facility) + \
        lpSum(distance[j][i]*serv_vars[(i,j)] for j in facility for i in customers)

# CONSTRAINTS
for i in customers:
    prob += lpSum(serv_vars[(i, j)] for j in facility) == demand[i]
for j in facility:
    prob += lpSum(serv_vars[(i, j)] for i in customers) <= capacity[j]*use_vars[j]

for i in customers:
    for j in facility:
        prob += serv_vars[(i, j)] <= demand[i]*use_vars[j]
        
#SOLVE
prob.solve()
print("Status:", LpStatus[prob.status])

# PRINT DECISION VARIABLES
TOL = 1e-5
for i in facility:
    if use_vars[i].varValue > TOL:
        print("Establish facility at site ", i)

for v in prob.variables():
    print(f"{v.name}={v.varValue}")
    
# OPTIMAL SOLUTION
print(f"The cost of production for one year = {value(prob.objective)}")

Status: Optimal
Establish facility at site  f2
Establish facility at site  f3
Service_(1,_'f1')=0.0
Service_(1,_'f2')=80.0
Service_(1,_'f3')=0.0
Service_(2,_'f1')=0.0
Service_(2,_'f2')=270.0
Service_(2,_'f3')=0.0
Service_(3,_'f1')=0.0
Service_(3,_'f2')=150.0
Service_(3,_'f3')=100.0
Service_(4,_'f1')=0.0
Service_(4,_'f2')=0.0
Service_(4,_'f3')=160.0
Service_(5,_'f1')=0.0
Service_(5,_'f2')=0.0
Service_(5,_'f3')=180.0
UseLocation_f1=0.0
UseLocation_f2=1.0
UseLocation_f3=1.0
The cost of production for one year = 5610.0


In [ ]:
from pulp import *

#SETS
nodes = [1, 2, 3, 4, 5]  # customers are nodes, they express demand
disks = ['T1', 'T2', 'T3'] # disks are facility, they offer service with limited capacity

#PARAMS

demand = {1:80, 2: 270, 3:250, 4: 160, 5: 180} # <- sizes in GB  # could detail files/objects to be read/written

latency = {'T1': 10e-3, 'T2': 100e-3, 'T3': 1000e-3} # latencies in s for seek/access, for hdd/tape may be statistical distrib

capacity = {"T1": 300, "T2": 500, "T3": 1e3} # in GB

bandwidth = {"T1": 10, "T2": 2, "T3": 1} # in GB/s <- could vary with access mode R/W Seq/Stride/Rand req_size

prob = LpProblem("DataPlacementTimeMin", LpMinimize)

y = LpVariable.dicts("Placement",
                         [(i, j) for i in disks for j in nodes], lowBound=0, cat='Binary') #lower band, >=0 #y(i, j)


x = LpVariable.dicts("DiskUse", disks, 0, 1, LpBinary) #x(i) indicates if disk i is accessed

# OBJECTIVE FUNCTION
prob += lpSum(latency[i]*x[i] for i in disks) + lpSum(demand[j]*y[(i, j)]/bandwidth[i] for i in disks for j in nodes) 
#prob += lpSum(demand[j]*y[(i, j)]/bandwidth[i] for i in disks for j in nodes) 
        # latency cost when using disk i       + transfer time 

# CONSTRAINTS
for j in nodes:
    # data served on disks equals 100% from node j, no duplicates!
    prob += lpSum(y[(i, j)] for i in disks) == 1   
for i in disks:
    # data put on disk i from all nodes is lower than its capacity
    prob += lpSum(y[(i, j)]*demand[j] for j in nodes) <= capacity[i]*x[i]


for j in nodes:
    for i in disks:
        # fraction of data served from node j to disk i is lower that demand of node j
        prob += y[(i, j)] <= demand[j]*x[i] 
        
#SOLVE
prob.solve()
print("Status:", LpStatus[prob.status])

# PRINT DECISION VARIABLES
TOL = 1e-6
for i in disks:
    if x[i].varValue > TOL:
        print("Establish connection with disk ", i)

for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
    
# OPTIMAL SOLUTION
print(f"The cost of this placement is  = {value(prob.objective)} seconds")

In [ ]:
from pulp import *

#SETS
nodes = [1, 2, 3, 4, 5]  # customers are nodes, they express demand
disks = ['T1', 'T2', 'T3'] # disks are facility, they offer service with limited capacity

#PARAMS

demand = {1:80, 2: 270, 3:250, 4: 160, 5: 180} # <- sizes in GB  # could detail files/objects to be read/written

latency = {'T1': 10e-3, 'T2': 100e-3, 'T3': 1000e-3} # latencies in s for seek/access, for hdd/tape may be statistical distrib

capacity = {"T1": 300, "T2": 500, "T3": 1e3} # in GB

bandwidth = {"T1": 10, "T2": 2, "T3": 1} # in GB/s <- could vary with access mode R/W Seq/Stride/Rand req_size

prob = LpProblem("DataPlacementTimeMin", LpMinimize)

y = LpVariable.dicts("Placement",
                         [(i, j) for i in disks for j in nodes], lowBound=0, cat='Binary') #lower band, >=0 #y(i, j)


x = LpVariable.dicts("DiskUse", disks, 0, 1, LpBinary) #x(i) indicates if disk i is accessed

# OBJECTIVE FUNCTION
prob += lpSum(latency[i]*x[i] for i in disks) + lpSum(demand[j]*y[(i, j)]/bandwidth[i] for i in disks for j in nodes) 
#prob += lpSum(demand[j]*y[(i, j)]/bandwidth[i] for i in disks for j in nodes) 
        # latency cost when using disk i       + transfer time 

# CONSTRAINTS
for j in nodes:
    # data served on disks equals 100% from node j, no duplicates!
    prob += lpSum(y[(i, j)] for i in disks) == 1   
for i in disks:
    # data put on disk i from all nodes is lower than its capacity
    prob += lpSum(y[(i, j)]*demand[j] for j in nodes) <= capacity[i]*x[i]


for j in nodes:
    for i in disks:
        # fraction of data served from node j to disk i is lower that demand of node j
        prob += y[(i, j)] <= demand[j]*x[i] 
        
#SOLVE
prob.solve()
print("Status:", LpStatus[prob.status])

# PRINT DECISION VARIABLES
TOL = 1e-6
for i in disks:
    if x[i].varValue > TOL:
        print("Establish connection with disk ", i)

for v in prob.variables():
    print(f"{v.name} = {v.varValue}")
    
# OPTIMAL SOLUTION
print(f"The cost of this placement is  = {value(prob.objective)} seconds")